In [1]:
import tensorflow as tf

In [2]:
from half_cheetah_env import HalfCheetahEnv

env = HalfCheetahEnv()

In [3]:
state = env.reset()

In [9]:
_num_random_action_selection = 4096
_horizon = 15
_state_dim = env.observation_space.shape[0]
_action_dim = env.action_space.shape[0]
_num_random_action_selection = 4096
_action_space_low = env.action_space.low
_action_space_high = env.action_space.high
print("_action_space_low:%s, _action_space_high:%s" % (_action_space_low, _action_space_high))
_state_dim

_action_space_low:[-1. -1. -1. -1. -1. -1.], _action_space_high:[1. 1. 1. 1. 1. 1.]


20

In [5]:
actions_sequences = tf.random_uniform([_num_random_action_selection, _horizon, _action_dim], 
                                      _action_space_low, _action_space_high, tf.float32)

In [ ]:
for i in range(_num_random_action_selection):
    for j in range(_horizon):
        action_for_state_i = tf.squeeze(tf.slice(actions_sequences, [i,j,0], [1,1,_action_dim]))
        print(action_for_state_i)

In [ ]:
with tf.Session() as sess:
    actions_sequences.eval()

In [8]:
with tf.Session() as sess:
    for i in range(_horizon):
        if i == 0:
            print(actions_sequences[:,i].eval().shape)

(4096, 6)


In [ ]:
def build_mlp(input_layer,
              output_dim,
              scope,
              n_layers=1,
              hidden_dim=500,
              activation=tf.nn.relu,
              output_activation=None,
              reuse=False):
    layer = input_layer
    with tf.variable_scope(scope, reuse=reuse):
        for _ in range(n_layers):
            layer = tf.layers.dense(layer, hidden_dim, activation=activation)
        layer = tf.layers.dense(layer, output_dim, activation=output_activation)
    return layer

In [ ]:
# reuse scope for variables
normalized_state_action = tf.placeholder(tf.float32, shape=[None, _action_dim + _state_dim])
normalized_next_state_diff_pred = build_mlp(normalized_state_action, _state_dim, scope="f_func")
v1 = build_mlp(normalized_state_action, _state_dim, scope="f_func", reuse=True)
tf.get_default_graph()